### Imports and version checks

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

In [2]:
print("You are on TF{}.".format(tf.__version__))

gpus = tf.config.experimental.list_physical_devices('GPU')
if len(gpus) == 0:
    print("You are not GPU accelerated.")
else:
    for gpu in gpus:
        print("Name:", gpu.name, "  Type:", gpu.device_type)

You are on TF2.3.1.
You are not GPU accelerated.


### Load and preprocess data, create examples

In [3]:
path = "infinite_jest_text.txt"

with open(path, "r") as f:
    text = f.read()
    
text = text.lower().replace("\n", " ")

unique_chars = sorted(list(set(text)))

idx_to_char = dict((i,c) for (i,c) in enumerate(unique_chars))
char_to_idx = dict((c, i) for (i, c) in enumerate(unique_chars))

Now onto creating training examples out of this input data.

For this particular task, we don't need to worry about validation and test sets. We always predict the next character for a given sentence.

In [4]:
maxlen = 40
stride = 3
sentences = []
next_chars = []

for i in range(0, len(text)-maxlen, stride):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])

Let's take a look at a pair of a sentence + its next character.

In [5]:
print("Sentence: {}\nNext character: {}".format(sentences[25], next_chars[25]))

Sentence:  undergarment 1 april — year of the tuck
Next character: s


We have sentences and the character that follows them. Now, we need to encode these into labelled training examples.

My thinking on the shape of `x` is:
- we take each sentence,
- we take each character in the sentence (40),
- we encode this character in a one-hot vector whose size is equal to however many unique characters we have.   

My thinking on the shape of `y` is:
- we take each sentence,
- we encode the character that follows it in a one-hot vector as above.

In [6]:
x = np.zeros((len(sentences), maxlen, len(unique_chars)))
y = np.zeros((len(sentences), len(unique_chars)))

# Let's now go through our sentences and characters and encode examples.
for sentence_index, sentence in enumerate(sentences):
    for char_index, char in enumerate(sentence):
        x[sentence_index, char_index, char_to_idx[char]] = 1
    y[sentence_index, char_to_idx[next_chars[sentence_index]]] = 1

Let's see what one input sentence and one output character look like encoded. 

In [7]:
print("One input char: {}".format(x[0][0]))
print("One output char: {}".format(y[0]))

One input char: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
One output char: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]


To get an even better idea of how this works, let's reconstruct the sentence and the character that follows it.

In [8]:
which_sentence = 0
chars = []
for char_vector in x[which_sentence]:
    chars.append(unique_chars[np.argmax(char_vector)])
    
print("Input sentence: {}".format("".join(chars)))
print("Next char: {}".format(unique_chars[np.argmax(y[which_sentence])]))

Input sentence: infinite jest by david foster wallace ye
Next char: a


Which is correct, because the first sentence reads:

```INFINITE JEST by David Foster Wallace
YEAR OF GLAD```

In [9]:
print(x.shape)

(1068040, 40, 103)


### Create a model, compile it

In [13]:
batch_size = 128

# Creating the model is the simplest part of this notebook.
model = keras.Sequential(
[
    # FIXME: what's the dimension of this input supposed to be?
    keras.layers.Input((x.shape[0], x.shape[1]), batch_size, name="Input"), 
    keras.layers.LSTM(128, name="LSTM"),
    keras.layers.Dense(len(unique_chars), activation="softmax", name="Dense")
])

In [14]:
optimizer = keras.optimizers.Adam()

# what should the loss be? what is each loss good for?
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [15]:
model.fit(x, y, epochs=1)

ValueError: in user code:

    /Users/inwaves/anaconda3/envs/my_first_env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Users/inwaves/anaconda3/envs/my_first_env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/inwaves/anaconda3/envs/my_first_env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/inwaves/anaconda3/envs/my_first_env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/inwaves/anaconda3/envs/my_first_env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
   

### Create a function for sampling/generating sequences from a seed sequence using a (partially) trained model